In [1]:
import numpy as np
import pandas as pd
import ast


In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [6]:
movies.dropna(inplace=True)

In [7]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [8]:
movies['genres']=movies['genres'].apply(convert)

movies['keywords']=movies['keywords'].apply(convert)

In [9]:
def convert3(obj):
    l = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
           l.append(i['name'])
           count+=1
        else:
            break
    return l

In [10]:
movies['cast']=movies['cast'].apply(convert3)

In [11]:
def fetchdir(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [12]:

movies['crew']=movies['crew'].apply(fetchdir)


In [13]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [14]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])


In [15]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [17]:
newdf = movies[['movie_id','title','tags']]

In [18]:
newdf['tags']=newdf['tags'].apply(lambda x:" ".join(x))


C:\Users\pravesh trivedi\AppData\Local\Temp\ipykernel_13040\1883337987.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags']=newdf['tags'].apply(lambda x:" ".join(x))


In [19]:
newdf['tags']=newdf['tags'].apply(lambda x:x.lower())

C:\Users\pravesh trivedi\AppData\Local\Temp\ipykernel_13040\3394916160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags']=newdf['tags'].apply(lambda x:x.lower())


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [21]:
vector = cv.fit_transform(newdf['tags']).toarray()

In [22]:
import nltk

In [23]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [24]:
def stem(text):
    y =  []
    for i in text.split():
        y.append(ps.steam(i))
    return " ",join(y)

In [25]:
newdf['tags']=newdf['tags'].apply(stem)


AttributeError: 'PorterStemmer' object has no attribute 'steam'

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [27]:
vector = cv.fit_transform(newdf['tags']).toarray()

In [28]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
similarity=cosine_similarity(vector)


In [31]:
def recommand(movie):
    movie_index = newdf[newdf['title'] == movie].index[0]
    distances = similarity[movie_index]
    movieslist = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movieslist:
        print(newdf.iloc[i[0]].title)

In [32]:
import pickle

In [34]:
pickle.dump(newdf,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [36]:
pickle.dump(newdf.to_dict(),open('movie_dict.pkl','wb'))